In [5]:
##Step1 ranking
import os
import subprocess
file_name="MyFile8"
ranking_method="randomForests"
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+file_name+".txt","-load","trainedModels/"+ranking_method+".txt","-norm","zscore","-indri","myNewRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)


Discard orig. features
Model file: trainedModels/randomForests.txt
Feature normalization: zscore
Model:  Random Forests
\rReading feature file [uploadedFiles/MyFile8.txt]: 0... \rReading feature file [uploadedFiles/MyFile8.txt]: 0... \rReading feature file [uploadedFiles/MyFile8.txt]... [Done.]            
(11 ranked lists, 12723 entries read)


In [24]:
#Step2 ranking(top-n)
#get top-n tweets id
n=150
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
counter=0
top_n={}
for line in lines:
    line_data=line.split(" ")
    print(line_data[4])
    qid=line_data[0]
    if qid not in top_n:
        top_n[qid]=[]
        counter=0
    if counter<n:
        top_n[qid].append(line_data[4])
    counter=counter+1

32_of_4_Years_of_FAN
57_of_4_Years_of_FAN
26_of_4_Years_of_FAN
69_of_4_Years_of_FAN
80_of_4_Years_of_FAN
45_of_4_Years_of_FAN
81_of_4_Years_of_FAN
107_of_4_Years_of_FAN
55_of_4_Years_of_FAN
103_of_4_Years_of_FAN
52_of_4_Years_of_FAN
71_of_4_Years_of_FAN
82_of_4_Years_of_FAN
68_of_4_Years_of_FAN
149_of_4_Years_of_FAN
35_of_4_Years_of_FAN
66_of_4_Years_of_FAN
116_of_4_Years_of_FAN
115_of_4_Years_of_FAN
88_of_4_Years_of_FAN
67_of_4_Years_of_FAN
111_of_4_Years_of_FAN
44_of_4_Years_of_FAN
61_of_4_Years_of_FAN
10_of_4_Years_of_FAN
129_of_4_Years_of_FAN
46_of_4_Years_of_FAN
102_of_4_Years_of_FAN
79_of_4_Years_of_FAN
109_of_4_Years_of_FAN
8_of_4_Years_of_FAN
47_of_4_Years_of_FAN
95_of_4_Years_of_FAN
74_of_4_Years_of_FAN
42_of_4_Years_of_FAN
41_of_4_Years_of_FAN
89_of_4_Years_of_FAN
76_of_4_Years_of_FAN
127_of_4_Years_of_FAN
54_of_4_Years_of_FAN
117_of_4_Years_of_FAN
78_of_4_Years_of_FAN
40_of_4_Years_of_FAN
70_of_4_Years_of_FAN
94_of_4_Years_of_FAN
135_of_4_Years_of_FAN
83_of_4_Years_of_FAN
86

523_of_coronavirus
491_of_coronavirus
493_of_coronavirus
665_of_coronavirus
499_of_coronavirus
510_of_coronavirus
617_of_coronavirus
501_of_coronavirus
520_of_coronavirus
525_of_coronavirus
522_of_coronavirus
682_of_coronavirus
553_of_coronavirus
521_of_coronavirus
482_of_coronavirus
487_of_coronavirus
497_of_coronavirus
505_of_coronavirus
488_of_coronavirus
509_of_coronavirus
508_of_coronavirus
524_of_coronavirus
496_of_coronavirus
506_of_coronavirus
585_of_coronavirus
503_of_coronavirus
515_of_coronavirus
504_of_coronavirus
507_of_coronavirus
582_of_coronavirus
546_of_coronavirus
517_of_coronavirus
697_of_coronavirus
516_of_coronavirus
575_of_coronavirus
530_of_coronavirus
626_of_coronavirus
594_of_coronavirus
568_of_coronavirus
705_of_coronavirus
599_of_coronavirus
544_of_coronavirus
598_of_coronavirus
699_of_coronavirus
560_of_coronavirus
548_of_coronavirus
532_of_coronavirus
628_of_coronavirus
613_of_coronavirus
611_of_coronavirus
603_of_coronavirus
618_of_coronavirus
637_of_coron

142_of_economical_crisis
149_of_economical_crisis
146_of_economical_crisis
150_of_economical_crisis
155_of_economical_crisis
154_of_economical_crisis
158_of_economical_crisis
153_of_economical_crisis
159_of_economical_crisis
174_of_economical_crisis
161_of_economical_crisis
160_of_economical_crisis
157_of_economical_crisis
156_of_economical_crisis
168_of_economical_crisis
169_of_economical_crisis
162_of_economical_crisis
165_of_economical_crisis
173_of_economical_crisis
166_of_economical_crisis
164_of_economical_crisis
167_of_economical_crisis
172_of_economical_crisis
171_of_economical_crisis
163_of_economical_crisis
170_of_economical_crisis
175_of_economical_crisis
177_of_economical_crisis
182_of_economical_crisis
181_of_economical_crisis
183_of_economical_crisis
178_of_economical_crisis
176_of_economical_crisis
184_of_economical_crisis
179_of_economical_crisis
180_of_economical_crisis
189_of_economical_crisis
186_of_economical_crisis
187_of_economical_crisis
185_of_economical_crisis


1441_of_INDvAUS
1116_of_INDvAUS
1524_of_INDvAUS
1356_of_INDvAUS
1273_of_INDvAUS
1118_of_INDvAUS
1481_of_INDvAUS
1348_of_INDvAUS
1696_of_INDvAUS
1835_of_INDvAUS
1624_of_INDvAUS
1068_of_INDvAUS
1271_of_INDvAUS
1418_of_INDvAUS
1376_of_INDvAUS
1646_of_INDvAUS
1513_of_INDvAUS
1243_of_INDvAUS
1476_of_INDvAUS
1694_of_INDvAUS
1689_of_INDvAUS
1642_of_INDvAUS
1904_of_INDvAUS
1445_of_INDvAUS
1047_of_INDvAUS
1331_of_INDvAUS
1383_of_INDvAUS
1384_of_INDvAUS
1334_of_INDvAUS
1764_of_INDvAUS
1634_of_INDvAUS
1682_of_INDvAUS
1508_of_INDvAUS
1440_of_INDvAUS
1661_of_INDvAUS
1593_of_INDvAUS
1385_of_INDvAUS
1075_of_INDvAUS
1327_of_INDvAUS
1310_of_INDvAUS
1705_of_INDvAUS
1109_of_INDvAUS
1262_of_INDvAUS
2031_of_INDvAUS
1288_of_INDvAUS
1656_of_INDvAUS
1212_of_INDvAUS
1233_of_INDvAUS
1072_of_INDvAUS
1265_of_INDvAUS
1379_of_INDvAUS
1594_of_INDvAUS
1708_of_INDvAUS
1256_of_INDvAUS
1585_of_INDvAUS
2067_of_INDvAUS
1320_of_INDvAUS
1190_of_INDvAUS
1260_of_INDvAUS
1649_of_INDvAUS
1494_of_INDvAUS
1647_of_INDvAUS
1655_of_

3222_of_INDvAUS
3035_of_INDvAUS
3368_of_INDvAUS
3245_of_INDvAUS
3417_of_INDvAUS
3064_of_INDvAUS
3018_of_INDvAUS
3072_of_INDvAUS
3036_of_INDvAUS
3037_of_INDvAUS
3145_of_INDvAUS
3246_of_INDvAUS
3538_of_INDvAUS
3127_of_INDvAUS
3230_of_INDvAUS
3079_of_INDvAUS
3207_of_INDvAUS
3154_of_INDvAUS
3335_of_INDvAUS
3244_of_INDvAUS
3143_of_INDvAUS
3176_of_INDvAUS
3271_of_INDvAUS
3139_of_INDvAUS
3057_of_INDvAUS
3302_of_INDvAUS
3086_of_INDvAUS
3447_of_INDvAUS
3286_of_INDvAUS
3150_of_INDvAUS
3027_of_INDvAUS
3164_of_INDvAUS
3126_of_INDvAUS
3061_of_INDvAUS
3223_of_INDvAUS
3102_of_INDvAUS
3296_of_INDvAUS
3122_of_INDvAUS
3706_of_INDvAUS
3337_of_INDvAUS
3345_of_INDvAUS
3052_of_INDvAUS
3136_of_INDvAUS
3097_of_INDvAUS
3316_of_INDvAUS
3053_of_INDvAUS
3166_of_INDvAUS
3294_of_INDvAUS
3261_of_INDvAUS
3333_of_INDvAUS
3180_of_INDvAUS
3040_of_INDvAUS
3054_of_INDvAUS
3000_of_INDvAUS
2996_of_INDvAUS
2998_of_INDvAUS
3326_of_INDvAUS
3475_of_INDvAUS
3306_of_INDvAUS
3162_of_INDvAUS
3218_of_INDvAUS
3388_of_INDvAUS
3224_of_

76_of_kohli
80_of_kohli
52_of_kohli
77_of_kohli
78_of_kohli
74_of_kohli
61_of_kohli
26_of_kohli
72_of_kohli
29_of_kohli
79_of_kohli
60_of_kohli
30_of_kohli
46_of_kohli
75_of_kohli
81_of_kohli
11_of_kohli
63_of_kohli
62_of_kohli
48_of_kohli
47_of_kohli
49_of_kohli
12_of_kohli
50_of_kohli
82_of_kohli
64_of_kohli
65_of_kohli
88_of_kohli
8_of_kohli
97_of_kohli
87_of_kohli
67_of_kohli
51_of_kohli
66_of_kohli
83_of_kohli
90_of_kohli
91_of_kohli
92_of_kohli
84_of_kohli
89_of_kohli
95_of_kohli
93_of_kohli
100_of_kohli
94_of_kohli
101_of_kohli
96_of_kohli
98_of_kohli
99_of_kohli
103_of_kohli
102_of_kohli
104_of_kohli
111_of_kohli
105_of_kohli
108_of_kohli
107_of_kohli
106_of_kohli
110_of_kohli
114_of_kohli
109_of_kohli
115_of_kohli
123_of_kohli
112_of_kohli
117_of_kohli
137_of_kohli
136_of_kohli
124_of_kohli
138_of_kohli
118_of_kohli
135_of_kohli
139_of_kohli
119_of_kohli
113_of_kohli
116_of_kohli
141_of_kohli
140_of_kohli
130_of_kohli
120_of_kohli
131_of_kohli
132_of_kohli
134_of_kohli
150_of_

531_of_narendra_modi
520_of_narendra_modi
538_of_narendra_modi
524_of_narendra_modi
507_of_narendra_modi
505_of_narendra_modi
536_of_narendra_modi
496_of_narendra_modi
519_of_narendra_modi
517_of_narendra_modi
526_of_narendra_modi
509_of_narendra_modi
522_of_narendra_modi
502_of_narendra_modi
528_of_narendra_modi
501_of_narendra_modi
518_of_narendra_modi
510_of_narendra_modi
506_of_narendra_modi
500_of_narendra_modi
497_of_narendra_modi
511_of_narendra_modi
523_of_narendra_modi
499_of_narendra_modi
493_of_narendra_modi
504_of_narendra_modi
514_of_narendra_modi
532_of_narendra_modi
533_of_narendra_modi
529_of_narendra_modi
527_of_narendra_modi
534_of_narendra_modi
525_of_narendra_modi
513_of_narendra_modi
516_of_narendra_modi
515_of_narendra_modi
503_of_narendra_modi
535_of_narendra_modi
494_of_narendra_modi
512_of_narendra_modi
530_of_narendra_modi
539_of_narendra_modi
537_of_narendra_modi
543_of_narendra_modi
540_of_narendra_modi
541_of_narendra_modi
573_of_narendra_modi
616_of_narend

645_of_netflix
673_of_netflix
842_of_netflix
693_of_netflix
699_of_netflix
695_of_netflix
688_of_netflix
698_of_netflix
907_of_netflix
658_of_netflix
696_of_netflix
723_of_netflix
853_of_netflix
716_of_netflix
691_of_netflix
694_of_netflix
711_of_netflix
709_of_netflix
677_of_netflix
669_of_netflix
712_of_netflix
726_of_netflix
786_of_netflix
756_of_netflix
697_of_netflix
759_of_netflix
725_of_netflix
728_of_netflix
829_of_netflix
783_of_netflix
798_of_netflix
710_of_netflix
706_of_netflix
781_of_netflix
764_of_netflix
720_of_netflix
743_of_netflix
734_of_netflix
777_of_netflix
774_of_netflix
884_of_netflix
692_of_netflix
888_of_netflix
719_of_netflix
668_of_netflix
782_of_netflix
871_of_netflix
779_of_netflix
775_of_netflix
745_of_netflix
770_of_netflix
705_of_netflix
729_of_netflix
739_of_netflix
672_of_netflix
741_of_netflix
905_of_netflix
768_of_netflix
735_of_netflix
862_of_netflix
671_of_netflix
748_of_netflix
740_of_netflix
744_of_netflix
746_of_netflix
788_of_netflix
794_of_net

406_of_Pandemic
346_of_Pandemic
407_of_Pandemic
433_of_Pandemic
418_of_Pandemic
428_of_Pandemic
434_of_Pandemic
417_of_Pandemic
398_of_Pandemic
399_of_Pandemic
424_of_Pandemic
458_of_Pandemic
452_of_Pandemic
453_of_Pandemic
435_of_Pandemic
443_of_Pandemic
413_of_Pandemic
419_of_Pandemic
440_of_Pandemic
460_of_Pandemic
423_of_Pandemic
468_of_Pandemic
455_of_Pandemic
448_of_Pandemic
502_of_Pandemic
388_of_Pandemic
462_of_Pandemic
505_of_Pandemic
467_of_Pandemic
486_of_Pandemic
473_of_Pandemic
463_of_Pandemic
522_of_Pandemic
438_of_Pandemic
493_of_Pandemic
402_of_Pandemic
465_of_Pandemic
469_of_Pandemic
528_of_Pandemic
538_of_Pandemic
531_of_Pandemic
454_of_Pandemic
477_of_Pandemic
482_of_Pandemic
544_of_Pandemic
390_of_Pandemic
512_of_Pandemic
520_of_Pandemic
446_of_Pandemic
546_of_Pandemic
518_of_Pandemic
543_of_Pandemic
392_of_Pandemic
474_of_Pandemic
550_of_Pandemic
408_of_Pandemic
529_of_Pandemic
552_of_Pandemic
464_of_Pandemic
461_of_Pandemic
445_of_Pandemic
441_of_Pandemic
506_of_P

In [31]:
#get details of top-n tweets and make new file for testing into second phase
file1 = open("uploadedFiles/"+file_name+".txt","r")
tweets={}
for x in file1:
    curr_row=x.split(" ")
    l=len(curr_row)
    doc_id=curr_row[l-1]
    doc_id=doc_id[:len(doc_id)-1]
    tweets[doc_id]=x
    
file1 = open("uploadedFiles/intermidiate_"+file_name+".txt","w")
for key in top_n.keys():
    for doc in top_n[key]:
        file1.write(tweets[doc])
file1.close()

In [33]:
#second phase ranking 
##Step1 ranking
import os
import subprocess
intermidiate_file_name="intermidiate_"+file_name
intermidiate_ranking_method="intermidiate_"+ranking_method
cmd=["java","-jar","RankLib-2.13.jar","-rank","uploadedFiles/"+intermidiate_file_name+".txt","-load","trainedModels/"+intermidiate_ranking_method+".txt","-norm","zscore","-indri","intermidiateRankedLists.txt"]
output=[]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
for line in proc.stdout.readlines():
    x=str(line)
    x=x[2:len(x)-3]
    x=x.replace('\\t',' ')
    print(x)
    output.append(x)


Discard orig. features
Model file: trainedModels/intermidiate_randomForests.txt
Feature normalization: zscore
Model:  Random Forests
\rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]: 0... \rReading feature file [uploadedFiles/intermidiate_MyFile8.txt]... [Done.]            
(11 ranked lists, 1650 entries read)


In [40]:
file = open("myNewRankedLists.txt","r")
lines = file.readlines()
rankList={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList:
        rankList[qid]=[]
    rankList[qid].append(line)
file.close()
file = open("intermidiateRankedLists.txt","r")
lines = file.readlines()
rankList2={}
for line in lines:
    qid=int(line.split(" ")[0])
    if qid not in rankList2:
        rankList2[qid]=[]
    rankList2[qid].append(line)
file.close()

for key in rankList2.keys():
    for i in range(0, len(rankList2[key])):
        rankList[key][i]=rankList2[key][i]
file=open("myNewRankedLists.txt","w")
for key in rankList.keys():
    for i in range(0, len(rankList[key])):
        file.write(rankList[key][i])
file.close()


In [3]:
score_data={}
label_data = {}
query={}
file1 = open("uploadedFiles/"+file_name+".txt","r")
for x in file1:
    curr_row=x.split(" ")
    qid=int(curr_row[1].split(":")[1])
    date=float(curr_row[14].split(":")[1])
    index2=int(curr_row[26].split("_")[0])-1
    if qid not in score_data:
        label_data[qid] = {}
        score_data[qid] = []
        query[qid]=curr_row[26].split("_")[2]
    label_data[qid][index2+1] = int(curr_row[0])
    score_data[qid].append([0,date,index2+1])
file1.close()

In [ ]:
f = open("myNewRankedLists.txt","r")
f1 = f.readlines()
ans = {}
for x in f1:
    y = x.split()
    value = y[len(y)-2]
    q_id = y[0]
    z = x.split('_')
    i = z[0]
    i = i.split()
    # print(i)
    i = i[4]
    for v in range(len(score_data[int(q_id)])):
        # print(score_data[int(q_id)][v][2])
        # print(i)
        if score_data[int(q_id)][v][2] == int(i):
            score_data[int(q_id)][v][0] = float(value)
            break
for id in score_data.keys():
    score_data[id].sort(key = lambda sub: (-sub[0], sub[1]))